In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [4]:
%gui qt

In [7]:
from spiketag.base import *

In [8]:
from spiketag.view import *

In [9]:
prb = probe(shank_no=1)
prb[0] = np.array([0,1,2])

In [10]:
prb.mapping[0] = np.array([-10,0])
prb.mapping[1] = np.array([10,0])
prb.mapping[2] = np.array([0,10])

In [11]:
prb.fs = 20000.
prb.n_ch = 3
prb.reorder_by_chip=False

In [10]:
prb.show()

/anaconda2/envs/py37/lib/python3.6/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated since IPython 4.0. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)


## MUA

In [14]:
mua = MUA(mua_filename='./cell_0109.bin', spk_filename='./cell_0109.spk.bin',
          probe=prb, binary_radix=11,
          cutoff=[-100,100], time_segs=np.array([[0, 40], [100., 200.]]) )

2018-12-31 20:24:57,554 - spiketag - INFO - #############  load data  ###################
2018-12-31 20:24:57,555 - spiketag - INFO - ./cell_0109.bin loaded, it contains: 
2018-12-31 20:24:57,555 - spiketag - INFO - 9280040.0 * 3 points (111360480 bytes) 
2018-12-31 20:24:57,556 - spiketag - INFO - 3 channels with sampling rate of 20000.0000 
2018-12-31 20:24:57,557 - spiketag - INFO - 464.002 secs (7.733 mins) of data
2018-12-31 20:24:57,557 - spiketag - INFO - #############################################
2018-12-31 20:24:57,641 - spiketag - INFO - processing folder: ./
2018-12-31 20:24:57,644 - spiketag - INFO - raw data have 9309 spks
2018-12-31 20:24:57,644 - spiketag - INFO - ----------------success------------------
2018-12-31 20:24:57,645 - spiketag - INFO -  


convert data from memmap to numpy with radix, elapsed time: 83.14204216003418 ms


In [15]:
mua.pivotal_pos

array([[   4131,    4458,    6466, ..., 9278006, 9278135, 9279424],
       [      1,       1,       1, ...,       1,       1,       1]],
      dtype=int32)

In [16]:
mua.pivotal_pos[0]

array([   4131,    4458,    6466, ..., 9278006, 9278135, 9279424],
      dtype=int32)

In [17]:
spks = np.fromfile('./cell_0109.spk.bin', dtype=np.int32)

In [18]:
spks[::2]

array([   4131,    4458,    6466, ..., 9278006, 9278135, 9279424],
      dtype=int32)

In [19]:
np.allclose(mua.pivotal_pos[0], spks[::2])

True

## Unit test for individual views

In [20]:
wview = wave_view(mua.data, spks=spks)
wview.show()

In [21]:
spk = mua.tospk()

2018-12-31 20:25:26,416 - spiketag - INFO - mua.tospk()
2018-12-31 20:25:27,224 - spiketag - INFO - group 0 delete 1.59500693481276%(46.0/2884.0) spks via cutoff
2018-12-31 20:25:27,225 - spiketag - INFO - ----------------success------------------
2018-12-31 20:25:27,226 - spiketag - INFO -  


In [22]:
mua.spk_times[0]/mua.fs

array([  0.20655,   0.2229 ,   0.3233 , ..., 199.63765, 199.6433 ,
       199.7441 ])

In [23]:
mua.t

array([0.0000000e+00, 5.0000000e-05, 1.0000000e-04, ..., 4.6400185e+02,
       4.6400190e+02, 4.6400195e+02])

In [24]:
time_segs = np.array([[0, 40], [100., 200.]])

In [25]:
mua.spk_times[0][np.logical_and(mua.spk_times[0]/mua.fs<time_segs[1][1], mua.spk_times[0]/mua.fs>time_segs[1][0])].shape

(1874,)

In [26]:
mua.spk_times[0].shape

(2838,)

In [27]:
def find_spk_in_time_seg(spk_times, time_segs):
    spk_times_in_range = []
    for time_seg in time_segs:
        spk_in_time_seg = spk_times[logical_and(spk_times<time_seg[1], spk_times>time_seg[0])]
        spk_times_in_range.append(spk_in_time_seg)
    return np.hstack(np.array(spk_times_in_range))

In [28]:
%%timeit 
spk_times = find_spk_in_time_seg(mua.spk_times[0]/mua.fs, time_segs)

35 µs ± 2.72 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [29]:
spk.spk[0].shape

(2838, 19, 3)

In [30]:
s = np.delete(spk.spk[0], [0,1,2], axis=0)

In [31]:
s.shape

(2835, 19, 3)

In [32]:
fet = spk.tofet()

2018-12-31 20:25:42,841 - spiketag - INFO - spk._tofet(group_id=0, method=weighted-pca, ncomp=6, whiten=False)
2018-12-31 20:25:42,842 - spiketag - INFO - ----------------success------------------
2018-12-31 20:25:42,842 - spiketag - INFO -  


In [33]:
clu = fet.toclu()

2018-12-31 20:25:43,641 - spiketag - INFO - clustering start with 1 cpus
2018-12-31 20:25:43,912 - spiketag - INFO - clustering finished, used 0.2695348262786865 sec


In [34]:
spkview = spike_view()

In [35]:
spkview.set_data(spk.spk[0], clu[0])

In [36]:
spkview.show()

In [37]:
fetview = scatter_3d_view()

In [38]:
fetview.set_data(fet.fet[0], clu[0])

In [39]:
fetview.show()

In [40]:
ampview = amplitude_view(fs=prb.fs, scale=1)

In [41]:
ampview.set_data(spk.spk[0], clu[0], mua.pivotal_pos[0])

In [42]:
ampview.show()

In [43]:
clu[0].membership.shape

(2838,)

In [44]:
treeview = ctree_view()

In [45]:
treeview.set_data(clu[0])

In [53]:
treeview.show()

In [46]:
clu[0]._extra_info['condensed_tree'].dtype

dtype([('parent', '<i8'), ('child', '<i8'), ('lambda_val', '<f8'), ('child_size', '<i8')])

In [47]:
clu[0]._extra_info['condensed_tree']

array([(2838, 2512,  1.72408129, 1), (2838, 2513,  1.97728322, 1),
       (2838, 1117,  2.42933043, 1), ..., (2840, 1943, 25.75185902, 1),
       (2840,  581, 25.75185902, 1), (2840, 2070, 25.75185902, 1)],
      dtype=[('parent', '<i8'), ('child', '<i8'), ('lambda_val', '<f8'), ('child_size', '<i8')])

In [48]:
treeview._whole_tree.dtype

dtype([('parent', '<i8'), ('child', '<i8'), ('lambda_val', '<f8'), ('child_size', '<i8')])

In [49]:
treeview._whole_tree

array([(2838, 2512,  1.72408129, 1), (2838, 2513,  1.97728322, 1),
       (2838, 1117,  2.42933043, 1), ..., (2840, 1943, 25.75185902, 1),
       (2840,  581, 25.75185902, 1), (2840, 2070, 25.75185902, 1)],
      dtype=[('parent', '<i8'), ('child', '<i8'), ('lambda_val', '<f8'), ('child_size', '<i8')])

In [50]:
treeview._vertices

array([[ 4.00631265, -5.05165458],
       [11.89539706, -5.05165458],
       [11.89539706, -5.05368293],
       ...,
       [ 7.93619524, -5.05035189],
       [ 7.93619524, -5.05165458],
       [ 0.01465961, -5.05165458]])

In [51]:
treeview._faces

array([[    0,     1,     2],
       [    0,     3,     2],
       [    4,     5,     6],
       ...,
       [10976, 10979, 10978],
       [10980, 10981, 10982],
       [10980, 10983, 10982]])

In [52]:
treeview._face_colors

array([[0.86666667, 0.38039216, 0.95686275, 1.        ],
       [0.86666667, 0.38039216, 0.95686275, 1.        ],
       [0.86666667, 0.38039216, 0.95686275, 1.        ],
       ...,
       [0.933333  , 0.866667  , 0.509804  , 1.        ],
       [0.933333  , 0.866667  , 0.509804  , 1.        ],
       [0.933333  , 0.866667  , 0.509804  , 1.        ]])

In [39]:
from vispy import scene

In [40]:
canvas = scene.SceneCanvas(keys='interactive')
view = canvas.central_widget.add_view()

In [41]:
mesh = scene.visuals.Mesh()
mesh.mode = 'triangles'

In [48]:
mesh.set_data(vertices=treeview._vertices, 
              faces=treeview._faces.astype(np.int64), 
              face_colors=treeview._face_colors)

In [49]:
view.camera = 'panzoom'

In [50]:
view.add(mesh)

In [51]:
canvas.show()

In [41]:
treeview._mesh

In [42]:
treeview.show()

In [37]:
treeview.show()

  File "/anaconda2/envs/py37/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/anaconda2/envs/py37/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/anaconda2/envs/py37/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/anaconda2/envs/py37/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/anaconda2/envs/py37/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/anaconda2/envs/py37/lib/python3.6/site-packages/tornado/platform/asyncio.py", line 132, in start
    self.asyncio_loop.run_forever()
  File "/anaconda2/envs/py37/lib/python3.6/asyncio/base_events.py", line 427, in run_forever
    self._run_once()
  File "/anaconda2/envs/py37/lib/python3.6/asyncio/base_events.py", line 1440, in _run_once
    handle._run()
  File "/anaconda2/envs

In [39]:
corview = correlogram_view(fs=prb.fs)

In [40]:
corview.set_data(clu[0], mua.pivotal_pos[0])

AssertionError: 

In [44]:
corview.show()

In [54]:
traceview = trace_view(fs=prb.fs, spklen=19)

In [55]:
traceview.set_data(mua.data, clu[0], mua.pivotal_pos[0])

In [56]:
traceview.show()

## Test clu function

In [39]:
_clu = clu[0]

In [43]:
_clu.merge([1,2])

In [100]:
_clu.move({2:[0,1,2,3,4,5,6,7,8,9,10]}, 0)

array([ 0,  1,  5,  8, 14, 15, 19, 20, 27, 29, 32])

## Unit test for QT5 Wrapper

In [5]:
from spiketag.mvc import MainModel, MainView

/anaconda2/lib/python2.7/site-packages/numba/decorators.py:146: RuntimeWarning: Caching is not available when the 'parallel' target is in use. Caching is now being disabled to allow execution to continue.
  warnings.warn(msg, RuntimeWarning)


In [12]:
model = MainModel('./cell_0109.bin', './cell_0109.spk.bin', prb, binary_radix=11)

2019-01-21 17:36:41,977 - spiketag - INFO - load mua data
2019-01-21 17:36:42,051 - spiketag - INFO - #############  load data  ###################
2019-01-21 17:36:42,053 - spiketag - INFO - ./cell_0109.bin loaded, it contains: 
2019-01-21 17:36:42,054 - spiketag - INFO - 9280040 * 3 points (111360480 bytes) 
2019-01-21 17:36:42,055 - spiketag - INFO - 3 channels with sampling rate of 20000.0000 
2019-01-21 17:36:42,057 - spiketag - INFO - 464.002 secs (7.733 mins) of data
2019-01-21 17:36:42,058 - spiketag - INFO - #############################################
2019-01-21 17:36:42,451 - spiketag - INFO - processing folder: ./
2019-01-21 17:36:42,454 - spiketag - INFO - raw data have 9309 spks
2019-01-21 17:36:42,455 - spiketag - INFO - ----------------success------------------
2019-01-21 17:36:42,456 - spiketag - INFO -  


convert data from memmap to numpy with radix, elapsed time: 391.514062881 ms


In [13]:
model.sort()

2019-01-21 17:36:45,164 - spiketag - INFO - extract spikes from pivital meta data
2019-01-21 17:36:45,166 - spiketag - INFO - mua.tospk()
2019-01-21 17:36:46,160 - spiketag - INFO - group 0 delete 0.0%(0.0/9309.0) spks via cutoff
2019-01-21 17:36:46,164 - spiketag - INFO - ----------------success------------------
2019-01-21 17:36:46,165 - spiketag - INFO -  
2019-01-21 17:36:46,167 - spiketag - INFO - grouping spike time
2019-01-21 17:36:46,168 - spiketag - INFO - extrat features with weighted-pca
2019-01-21 17:36:46,203 - spiketag - INFO - spk._tofet(group_id=0, method=weighted-pca, ncomp=6, whiten=False)
2019-01-21 17:36:46,205 - spiketag - INFO - ----------------success------------------
2019-01-21 17:36:46,206 - spiketag - INFO -  
2019-01-21 17:36:46,207 - spiketag - INFO - clustering with hdbscan
2019-01-21 17:36:46,209 - spiketag - INFO - clustering start with 24 cpus
2019-01-21 17:36:50,931 - spiketag - INFO - clustering finished, used 4.71980500221 sec
2019-01-21 17:36:50,943

In [14]:
view = MainView(prb)

In [22]:
view.show()

In [16]:
view.set_data(0, model.mua, model.spk[0], model.fet[0], model.clu[0])

In [33]:
view.spkview.cluster_mouse_on

1

In [23]:
model.clu[0].index_count

{0: 7857,
 1: 372,
 2: 54,
 3: 58,
 4: 71,
 5: 84,
 6: 50,
 7: 59,
 8: 56,
 9: 59,
 10: 56,
 11: 61,
 12: 378,
 13: 94}

In [24]:
model.clu[0].npts

9309

In [25]:
model.clu[0].nclu

14

In [26]:
model.clu[0]._method

'hdbscan'

In [27]:
model.clu[0]._extra_info

{'condensed_tree': array([(9309, 3883,  3.68506103, 1), (9309, 3877,  4.67542588, 1),
        (9309, 8202,  4.68442154, 1), ..., (9332, 8880, 76.33922628, 1),
        (9332, 7390, 76.33922628, 1), (9332, 8733, 76.33922628, 1)],
       dtype=[('parent', '<i8'), ('child', '<i8'), ('lambda_val', '<f8'), ('child_size', '<i8')]),
 'default_select_clusters': array([9310, 9314, 9315, 9316, 9318, 9320, 9324, 9325, 9326, 9329, 9330,
        9332, 9333])}

In [28]:
_labels = model.clu[0]._probmatrix.argmax(axis=1)

In [29]:
idx = np.where(model.clu[0]._probmatrix.max(axis=1) > 0.02)[0]

In [30]:
_labels = np.zeros((model.clu[0].npts,))
_labels[idx] = model.clu[0]._probmatrix.argmax(axis=1)[idx] + 1

In [31]:
model.clu[0].fill(_labels)

In [31]:
model.clu[0].selectlist.shape

(681,)

## dpgmm

In [68]:
from sklearn.mixture import BayesianGaussianMixture as DPGMM
dpgmm = DPGMM(
        n_components=15, covariance_type='full', weight_concentration_prior=1e-3,
        weight_concentration_prior_type='dirichlet_process', init_params="kmeans",
        max_iter=500, random_state=None, tol=0.05, verbose=True) # init can be "kmeans" or "random"


In [69]:
dpgmm.fit(model.fet[0])
labels = dpgmm.predict(model.fet[0])

Initialization 0
  Iteration 10
  Iteration 20
  Iteration 30
  Iteration 40
  Iteration 50
  Iteration 60
  Iteration 70
  Iteration 80
  Iteration 90
  Iteration 100
  Iteration 110
  Iteration 120
  Iteration 130
Initialization converged: True


In [70]:
model.clu[0].fill(labels)

In [72]:
model.clu[0].selectlist.shape

(700,)

## hdbscan

In [16]:
import hdbscan

In [126]:
hdbcluster = hdbscan.HDBSCAN(min_samples=10,
             min_cluster_size=25, 
             leaf_size=40,
#              alpha=10.5,
             gen_min_span_tree=False, 
             algorithm='boruvka_kdtree',
             core_dist_n_jobs=1,
             prediction_data=True,
             cluster_selection_method='leaf') #leaf         
clusterer = hdbcluster.fit(model.fet[0].astype(np.float64))
probmatrix = hdbscan.all_points_membership_vectors(clusterer)
labels = clusterer.labels_

In [63]:
view.show()

In [23]:
labels

array([-1,  4, -1, ..., -1, -1, -1])

In [127]:
model.clu[0].fill(clusterer.labels_+1)

In [134]:
idx = np.where(probmatrix.max(axis=1) > 0.05)[0]

In [135]:
_labels = np.zeros((model.clu[0].npts,))
_labels[idx] = probmatrix.argmax(axis=1)[idx] + 1

In [136]:
_labels

array([5., 3., 3., ..., 3., 3., 5.])

In [137]:
model.clu[0].fill(_labels)

In [185]:
model.clu[0].selectlist.shape

(705,)

In [24]:
ctree = model.clu[0]._extra_info['condensed_tree']

In [27]:
ctree.dtype

dtype([('parent', '<i8'), ('child', '<i8'), ('lambda_val', '<f8'), ('child_size', '<i8')])

In [117]:
ctree['parent']

12

In [45]:
targeted = model.clu[0][2]
targeted.shape

(446,)

In [112]:
np.unique(ctree[targeted]['parent'])

array([9309, 9310, 9311, 9312, 9313, 9314, 9315, 9316, 9317, 9318, 9319,
       9321])

In [108]:
model.clu[0][9312]

AssertionError: 9312 should in [0 1 2 3 4]

In [125]:
ctree[ctree['child']==9316]

array([(9314, 9316, 27.34693829, 23)],
      dtype=[('parent', '<i8'), ('child', '<i8'), ('lambda_val', '<f8'), ('child_size', '<i8')])

In [93]:
view.treeview._select_clusters

array([9310, 9312, 9314, 9315])

In [96]:
view.treeview._clu_tree

array([(9309, 9310,  8.59999543,  370), (9309, 9311,  8.59999543, 8724),
       (9311, 9312, 17.69651766,  446), (9311, 9313, 17.69651766, 5921),
       (9313, 9314, 24.1508309 , 2857), (9313, 9315, 24.1508309 ,   52),
       (9314, 9316, 27.34693829,   23), (9314, 9317, 27.34693829, 1743),
       (9317, 9318, 30.26568792,  985), (9317, 9319, 30.26568792,   66),
       (9318, 9320, 38.22736817,   18), (9318, 9321, 38.22736817,  210)],
      dtype=[('parent', '<i8'), ('child', '<i8'), ('lambda_val', '<f8'), ('child_size', '<i8')])

In [102]:
view.treeview._select_clusters

array([9310, 9312, 9314, 9315])

In [54]:
ctree['child'][targeted]

array([3877, 2830, 3285, 8905, 4111, 6607, 3473, 4678, 3453, 3411, 5796,
       8657, 5925, 8382, 6507, 5558, 7185, 8915, 4960, 7624, 4122, 4476,
       7117, 7730, 6609, 6036, 6035, 3288, 4654, 7915, 5486, 5149, 5752,
       6644,  194, 7107, 5160, 2882, 8906, 7331, 4823, 3505,  270, 1384,
       4886, 3219,  881, 8212, 6001, 5877, 3432, 2893, 8087, 2020,  980,
       4986, 5728, 8125, 8477, 1150, 8086,  863, 1807, 3779, 5628, 1711,
       1618, 7571, 3373, 4489, 7457, 8130, 2723, 3129, 8137, 8143, 3110,
       9101, 8892, 8627, 1225, 1806, 7935, 2787, 4321, 1040, 7822, 6229,
       7389, 6642, 5591, 6436, 2347, 3554, 2298, 4849, 2653, 2354, 7348,
       6501, 9292,  275, 8342,  387, 6834, 9244, 6066, 8887, 4328, 9260,
       9002, 1603, 6562, 3468, 1647, 3888, 8197, 2105, 8285, 4254, 2770,
        962, 8038, 5120, 9133,  732, 3721, 3659, 3269,  657, 6266, 2554,
       5783,  731, 7817, 7988, 1050,  901, 2793, 7258, 4801, 4778,  283,
        817,  768, 1425, 2992, 5505, 5452, 5657, 60

In [79]:
ctree[ctree['child_size'] > 1]

array([(9309, 9310,  8.59999543,  370), (9309, 9311,  8.59999543, 8724),
       (9311, 9312, 17.69651766,  446), (9311, 9313, 17.69651766, 5921),
       (9313, 9314, 24.1508309 , 2857), (9313, 9315, 24.1508309 ,   52),
       (9314, 9316, 27.34693829,   23), (9314, 9317, 27.34693829, 1743),
       (9317, 9318, 30.26568792,  985), (9317, 9319, 30.26568792,   66),
       (9318, 9320, 38.22736817,   18), (9318, 9321, 38.22736817,  210)],
      dtype=[('parent', '<i8'), ('child', '<i8'), ('lambda_val', '<f8'), ('child_size', '<i8')])

In [46]:
ctree['lambda_val'][targeted]

array([ 3.76739517,  5.04050012,  5.06830131,  5.8088414 ,  5.81410217,
        6.62878933,  6.98132978,  7.28553743,  7.56710562,  7.66982786,
        8.03325103,  8.2531465 ,  8.96183654,  8.62872868,  9.11729181,
       10.0781649 , 10.98204098,  8.99939802, 11.72145135,  9.30157334,
       12.35833899,  9.45787805,  9.68819657,  9.87248173,  9.94101382,
       14.94272754, 16.00258994, 10.49652308, 10.55078833, 16.46198266,
       10.6934489 , 17.58396423, 11.13723658, 11.40943118, 11.53267058,
       11.54769561, 12.15373353, 12.36137397, 12.4191623 , 12.74415617,
       12.97999923, 12.9817445 , 13.8828103 , 14.08337721, 14.20722675,
       14.21359936, 14.78119942, 14.90207434, 14.91613003, 14.91782788,
       15.007126  , 15.01270161, 15.13116488, 15.13155285, 15.23442408,
       15.2942785 , 15.29494684, 15.37306517, 15.48341335, 15.48616845,
       15.67599533, 15.70690072, 15.74817555, 15.76661442, 15.76839468,
       15.77474438, 15.81989203, 15.88142388, 15.91036462, 15.92

In [30]:
ctree['child']

array([3883, 4984, 3884, ..., 1240,  685, 2343])

In [15]:
thr = -31.00

In [16]:
idx = np.where(model.spk[0].min(axis=1).min(axis=1)>thr)[0]

In [17]:
idx.shape

(5032,)

In [17]:
def delete_spk(model, spk_idx):
    model.mua.spk_times[0] = np.delete(model.mua.spk_times[0], spk_idx, axis=0)
    model.spk[0] = np.delete(model.spk[0], spk_idx, axis=0)
    model.fet[0] = model.spk._tofet(0, method='pca')
    model.cluster(group_id=0, method='hdbscan', fall_off_size=30) #, 
    view.set_data(0, model.mua, model.spk[0], model.fet[0], model.clu[0])

(4067,)
(2589,)
(2104,)


2018-06-07 16:28:50,182 - spiketag - DEBUG - no more undo
2018-06-07 16:28:50,532 - spiketag - DEBUG - no more undo
2018-06-07 16:28:50,760 - spiketag - DEBUG - no more undo
2018-06-07 16:28:50,946 - spiketag - DEBUG - no more undo
2018-06-07 16:28:51,118 - spiketag - DEBUG - no more undo
2018-06-07 16:28:51,289 - spiketag - DEBUG - no more undo
2018-06-07 16:28:51,453 - spiketag - DEBUG - no more undo
2018-06-07 16:28:51,632 - spiketag - DEBUG - no more undo
2018-06-07 16:28:51,843 - spiketag - DEBUG - no more undo
2018-06-07 16:28:52,147 - spiketag - DEBUG - no more undo
2018-06-07 16:28:52,437 - spiketag - DEBUG - no more undo
2018-06-07 16:28:52,671 - spiketag - DEBUG - no more undo
2018-06-07 16:28:52,932 - spiketag - DEBUG - no more undo
2018-06-07 16:28:53,445 - spiketag - DEBUG - no more undo
2018-06-07 16:28:53,680 - spiketag - DEBUG - no more undo
2018-06-07 16:28:53,938 - spiketag - DEBUG - no more undo
2018-06-07 16:28:54,136 - spiketag - DEBUG - no more undo


In [19]:
delete_spk(model, idx)

2018-06-07 16:16:50,296 - spiketag - DEBUG - ctree view expand clustier 4279 here


In [16]:
@view.ampview.clip.connect
def on_clip(thres):
    idx = np.where(model.spk[0].min(axis=1).min(axis=1)>thres)[0]
    print(idx.shape)
    delete_spk(model=model, spk_idx=idx)

  File "/anaconda2/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/anaconda2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/anaconda2/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/anaconda2/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 478, in start
    self.io_loop.start()
  File "/anaconda2/lib/python2.7/site-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
  File "/anaconda2/lib/python2.7/site-packages/tornado/ioloop.py", line 832, in start
    self._run_callback(self._callbacks.popleft())
  File "/anaconda2/lib/python2.7/site-packages/tornado/ioloop.py", line 605, in _run_callback
    ret = callback()
  File "/anaconda2/lib/python2.7/site-package

(4817,)


In [42]:
model.spk[0].shape

(1678, 19, 3)

In [39]:
model.fet = model.spk.tofet()

2018-06-07 11:33:46,742 - spiketag - INFO - spk._tofet(group_id=0, method=weighted-pca, ncomp=6, whiten=False)
2018-06-07 11:33:46,746 - spiketag - INFO - ----------------success------------------
2018-06-07 11:33:46,747 - spiketag - INFO -  


In [40]:
model.fet[0].shape

(1678, 6)

In [41]:
model.clu = model.fet.toclu()

2018-06-07 11:33:49,754 - spiketag - INFO - clustering finished, used 0.066253900528 sec


In [19]:
model.mua.spk_times[0].shape

(4740,)

In [16]:
view.ampview.poses[:, 1].min()

-146.53759765625

In [53]:
from spiketag.mvc.Control import controller

In [54]:
ctrl = controller(probe=prb, mua_filename='./cell_0109.bin', spk_filename='./cell_0109.spk.bin')

2018-12-28 20:20:37,338 - spiketag - INFO - load mua data
2018-12-28 20:20:37,401 - spiketag - INFO - #############  load data  ###################
2018-12-28 20:20:37,402 - spiketag - INFO - ./cell_0109.bin loaded, it contains: 
2018-12-28 20:20:37,403 - spiketag - INFO - 9280040.0 * 3 points (111360480 bytes) 
2018-12-28 20:20:37,404 - spiketag - INFO - 3 channels with sampling rate of 20000.0000 
2018-12-28 20:20:37,404 - spiketag - INFO - 464.002 secs (7.733 mins) of data
2018-12-28 20:20:37,405 - spiketag - INFO - #############################################
2018-12-28 20:20:37,470 - spiketag - INFO - processing folder: ./
2018-12-28 20:20:37,473 - spiketag - INFO - raw data have 9309 spks
2018-12-28 20:20:37,474 - spiketag - INFO - ----------------success------------------
2018-12-28 20:20:37,475 - spiketag - INFO -  


In [55]:
ctrl.sort()

2018-12-28 20:20:46,070 - spiketag - INFO - extract spikes from pivital meta data
2018-12-28 20:20:46,071 - spiketag - INFO - mua.tospk()
2018-12-28 20:20:46,378 - spiketag - INFO - group 0 delete 0.0%(0.0/9309.0) spks via cutoff
2018-12-28 20:20:46,381 - spiketag - INFO - ----------------success------------------
2018-12-28 20:20:46,381 - spiketag - INFO -  
2018-12-28 20:20:46,382 - spiketag - INFO - grouping spike time
2018-12-28 20:20:46,382 - spiketag - INFO - extrat features with weighted-pca
2018-12-28 20:20:46,404 - spiketag - INFO - spk._tofet(group_id=0, method=weighted-pca, ncomp=6, whiten=False)
2018-12-28 20:20:46,404 - spiketag - INFO - ----------------success------------------
2018-12-28 20:20:46,405 - spiketag - INFO -  
2018-12-28 20:20:46,406 - spiketag - INFO - clustering with hdbscan
2018-12-28 20:20:46,406 - spiketag - INFO - clustering start with 24 cpus
2018-12-28 20:20:50,116 - spiketag - INFO - clustering finished, used 3.7082719802856445 sec
2018-12-28 20:20:5

In [56]:
ctrl.show()

  File "/anaconda2/envs/py37/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/anaconda2/envs/py37/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/anaconda2/envs/py37/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/anaconda2/envs/py37/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/anaconda2/envs/py37/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/anaconda2/envs/py37/lib/python3.6/site-packages/tornado/platform/asyncio.py", line 132, in start
    self.asyncio_loop.run_forever()
  File "/anaconda2/envs/py37/lib/python3.6/asyncio/base_events.py", line 427, in run_forever
    self._run_once()
  File "/anaconda2/envs/py37/lib/python3.6/asyncio/base_events.py", line 1440, in _run_once
    handle._run()
  File "/anaconda2/envs

array([-19.429688, -18.166504, -17.284668, ..., -17.5     , -19.528809,
       -28.05664 ], dtype=float32)

In [14]:
model.mua.spk_times[0].shape

(9309,)

In [15]:
model.spk[0].shape

(9309, 19, 3)

In [16]:
model.fet[0].shape

(9309, 6)

In [28]:
import sys
from PyQt5.QtGui import *
from PyQt5.QtCore import *
from PyQt5.QtWidgets import *

class sorter(QWidget):

    def __init__(self):
        super(sorter, self).__init__()
        self.initUI()

    def initUI(self):

        hbox = QHBoxLayout(self)
        self.splitter1 = QSplitter(Qt.Horizontal)
#         textedit = QTextEdit()
#         self.splitter1.addWidget(self.topleft)
#         self.splitter1.addWidget(textedit)
#         self.splitter1.setSizes([100,200])
        self.splitter2 = QSplitter(Qt.Horizontal)
        self.splitter_fet = QSplitter(Qt.Vertical)

        self.splitter3 = QSplitter(Qt.Vertical)
        self.splitter3.addWidget(self.splitter1)
        self.splitter3.addWidget(self.splitter2)
#         self.splitter2.addWidget(self.bottom)

        hbox.addWidget(self.splitter3)

        self.setLayout(hbox)
        QApplication.setStyle(QStyleFactory.create('Cleanlooks'))

        self.setGeometry(300, 300, 300, 200)
        self.setWindowTitle('spiketag')
#         self.show()

    def set_data(self, mua, spk, fet, clu):
        ### init view and set_data
        self.spkview = spike_view()
        self.spkview.set_data(spk, clu)
        self.fetview0 = scatter_3d_view()
        self.fetview0.set_data(fet[:,[0,1,2]].copy(), clu)
        self.fetview1 = scatter_3d_view()
        self.fetview1.set_data(fet[:,[1,3,4]].copy(), clu)
        self.ampview = amplitude_view(fs=fs, scale=1)
        self.ampview.set_data(spk, clu, mua.pivotal_pos[0])
        self.treeview = ctree_view()
        self.treeview.set_data(clu)
        self.corview = correlogram_view(fs=fs)
        self.corview.set_data(clu, mua.pivotal_pos[0])
        self.traceview = trace_view(fs=mua.fs, spklen=spk.shape[1])
        self.traceview.set_data(mua.data, clu, mua.pivotal_pos[0])
#         self.traceview.locate_buffer = 2000
        
        ### put views into splitter
        self.splitter1.addWidget(self.traceview.native)
        self.splitter1.addWidget(self.splitter_fet)
        self.splitter_fet.addWidget(self.fetview0.native)
        self.splitter_fet.addWidget(self.fetview1.native)
        self.splitter1.addWidget(self.spkview.native)
        
        self.splitter2.addWidget(self.corview.native)
        self.splitter2.addWidget(self.treeview.native)
        self.splitter2.addWidget(self.ampview.native)


In [29]:
%gui qt

In [30]:
ex = sorter()

In [31]:
spk = mua.tospk()
fet = spk.tofet(method='pca', ncomp=9, whiten=False)
clu = fet.toclu(method='hdbscan')

2018-06-06 18:53:54,900 - spiketag - INFO - mua.tospk()
2018-06-06 18:53:54,909 - spiketag - INFO - ----------------success------------------
2018-06-06 18:53:54,910 - spiketag - INFO -  
2018-06-06 18:53:54,943 - spiketag - INFO - spk._tofet(groupNo=0, method=pca, ncomp=9, whiten=False)
2018-06-06 18:53:54,945 - spiketag - INFO - ----------------success------------------
2018-06-06 18:53:54,946 - spiketag - INFO -  
2018-06-06 18:53:55,855 - spiketag - INFO - clustering finished, used 0.906870126724 sec


In [32]:
ex.set_data(mua, spk.spk[1], fet.fet[1], clu[1])

KeyError: 1

In [66]:
ex.show()

In [70]:
ex.traceview.locate_buffer = 300

In [97]:
ex.fetview0.set_data(fet.fet[1][:,[1,3,4]], clu[1])
ex.update()

ERROR: Invoking <bound method trace_view.update_cursor of <trace_view (PyQt5) at 0x1c4774b690>> repeat 8192
ERROR: Invoking <bound method trace_view.update_cursor of <trace_view (PyQt5) at 0x1c4a0aa110>> repeat 8192


In [20]:
gclu = clu[1]

In [21]:
gclu.selectlist

array([ 848, 2681, 2795, 2899, 2951, 3162, 3521, 4018, 4031, 4074, 4169,
       4224, 4237, 4486, 4489, 4648, 5064, 5078, 5823, 5966, 5967, 6047,
       6049, 6070, 6610, 7207, 7319, 7919, 8057, 8167, 8378, 8602, 8679,
       9240])

2018-03-11 15:08:33,804 - spiketag - DEBUG - ctree view expand clustier 9311 here
2018-03-11 15:08:51,809 - spiketag - DEBUG - ctree view expand clustier 9313 here


In [21]:
gclu._registered_func_name('select')

['spiketag.view.spike_view.on_select_id121320795744',
 'spiketag.view.scatter_3d_view.on_select_id121444857928',
 'spiketag.view.scatter_3d_view.on_select_id121449105600',
 'spiketag.view.amplitude_view.on_select_id121452650320',
 'spiketag.view.trace_view.on_select_id121529220464']

2018-03-11 14:14:28,666 - spiketag - DEBUG - ctree view collapse cluster 9309 here
2018-03-11 14:14:29,401 - spiketag - DEBUG - ctree view collapse cluster 9309 here
2018-03-11 14:14:31,183 - spiketag - DEBUG - ctree view collapse cluster 9309 here
2018-03-11 14:14:32,756 - spiketag - DEBUG - ctree view collapse cluster 9311 here
2018-03-11 14:14:34,080 - spiketag - DEBUG - ctree view collapse cluster 9311 here
2018-03-11 14:14:34,932 - spiketag - DEBUG - ctree view collapse cluster 9309 here
2018-03-11 14:14:35,486 - spiketag - DEBUG - ctree view collapse cluster 9311 here
2018-03-11 14:14:36,014 - spiketag - DEBUG - ctree view collapse cluster 9311 here
2018-03-11 14:14:36,999 - spiketag - DEBUG - ctree view collapse cluster 9309 here
2018-03-11 14:14:37,535 - spiketag - DEBUG - ctree view collapse cluster 9311 here
2018-03-11 14:14:39,278 - spiketag - DEBUG - ctree view expand clustier 9312 here
2018-03-11 14:14:43,798 - spiketag - DEBUG - ctree view collapse cluster 9311 here
2018-